In [1]:
# pyenv activate cw
import sys
import time
import os
import numpy as np
import chipwhisperer as cw
from tqdm.notebook import tqdm
import serial
import matplotlib.pyplot as plt

ser = 0

In [6]:
# disable logging
cw.set_all_log_levels(cw.logging.CRITICAL)

# ------------------------------
# Initiate Connection to the Chipwhisperer Lite Kit
print("Connecting to CW_Lite.")
scope = cw.scope()
try:
    if not scope.connectStatus:
        scope.con()
except NameError:
    scope = cw.scope()
print("INFO: Found ChipWhisperer😍")
time.sleep(0.05)
# scope.default_setup()

Connecting to CW_Lite.
INFO: Found ChipWhisperer😍


In [9]:
target = cw.target(scope)
# Original attack done with 100 MHz clock - can be helpful to run this
# 2x faster to get better resolution, which seems useful for glitching certain boards.
# But if you want to use DPA you need to leave this set to '1'
freq_multiplier = 1

#Initial Setup
scope.adc.samples = 10000
scope.adc.offset = 0
scope.clock.adc_src = "clkgen_x1"
scope.trigger.triggers = "tio4"

# this value is for CW-Lite/Pro; for CW-Husky, refer to Fault 1_1
scope.glitch.width = 40
scope.io.tio1 = "serial_rx"
scope.io.tio2 = "serial_tx"
scope.adc.basic_mode = "rising_edge"
scope.clock.clkgen_freq = 100000000 * freq_multiplier
scope.glitch.clk_src = "clkgen"
scope.glitch.trigger_src = "ext_single"
scope.glitch.output = "enable_only"

# For use with the ChipShouter Pico

# Disable the glitch mosfet and route the glitch signal to hs2
# Connect HS2 to the ChipShouter Pico HVP pin
scope.io.glitch_lp = False
scope.io.glitch_hp = False
scope.io.hs2 = None
scope.io.hs2 = "glitch"
# scope.glitch.ext_offset = 300         # Glitch offset from the external trigger (in cycles of the main CW clock)
# scope.glitch.repeat = 100             # You might want to try different values for this parameter

# Set tio3 to be an input
# Connect tio3 to the CHG pin, this allows us to check if the ChipShouter is charged
scope.io.tio3 = 'high_z'
print(scope.io.tio_states[2])

1


In [10]:
# Simple loop that blocks until the ChipShouter is charged
def wait_for_hv():
    while scope.io.tio_states[2] != 0:
        time.sleep(0.1)

In [11]:
# A very basic class to interact with the ChipShouter PicoEMP

class ChipShouterPicoEMP:
    def __init__(self, port='/dev/ttyACM1'):
        self.pico = serial.Serial(port, 115200)
        
        self.pico.write(b'\r\n')
        time.sleep(0.1)
        ret = self.pico.read(self.pico.in_waiting)
        
        if b'PicoEMP Commands' in ret:
            print('Connected to ChipSHOUTER PicoEMP!')
        else:
            raise OSError('Could not connect to ChipShouter PicoEMP :(')
        

    def disable_timeout(self):
        self.pico.write(b'disable_timeout\r\n')
        time.sleep(1)
        assert b'Timeout disabled!' in self.pico.read(self.pico.in_waiting)

        
    def arm(self):
        self.pico.write(b'arm\r\n')
        time.sleep(1)
        assert b'Device armed' in self.pico.read(self.pico.in_waiting)

        
    def disarm(self):
        self.pico.write(b'disarm\r\n')
        time.sleep(1)
        assert b'Device disarmed!' in self.pico.read(self.pico.in_waiting)

        
    def external_hvp(self):
        self.pico.write(b'external_hvp\r\n')
        time.sleep(1)
        assert b'External HVP mode active' in self.pico.read(self.pico.in_waiting)

        
    def print_status(self):
        self.pico.write(b'status\r\n')
        time.sleep(1)
        print(self.pico.read(self.pico.in_waiting).decode('utf-8'))
        
    
    def setup_external_control(self):
        self.disable_timeout()
        self.external_hvp()
        self.print_status()

In [12]:
# Change the serial port if needed!
pico = ChipShouterPicoEMP('/dev/ttyACM0')
pico.setup_external_control()

Connected to ChipSHOUTER PicoEMP!
status
Status:
- Disarmed
- Not charged
- Timeout disabled
- HVP external

[status] > 



In [13]:
# Simple function to reset the target microcontroller
def reset_dut(delay=0.1):
    scope.io.nrst = 'low'
    scope.io.target_pwr = False
    time.sleep(delay)
    scope.io.target_pwr = True
    scope.io.nrst = 'high'
    time.sleep(0.05)

In [ ]:
scope.glitch.repeat = 35
offsets = np.arange(0, 500, 20)
repeats = 10
crashes = 0
faults = 0

pico.arm()

for offset in tqdm(range(len(offsets))):
    scope.glitch.ext_offset = offsets[offset]
    
    for i in range(repeats):
        wait_for_hv() # Wait for the ChipShouter to be charged
        
        scope.arm()
        ser.write(bytes([0xAA]))
        time.sleep(0.1)
        ret = ser.read(ser.in_waiting)
        
        if ret != b"\x10'\x00\x00":
            if ret == b'':
                crashes += 1
                print('.', end = '')
            else:
                faults += 1
                print('Fault?!', int.from_bytes(ret, 'little'), ret.hex(), offsets[offset])
            
            thorough_reset_dut() 
            
    if offset % 20 == 0:
        print('\n', crashes, faults)
                
pico.disarm()

total = len(offsets)*repeats
print("\nTotal # attempts:", total) 
print("Total # faults: %d (%f%%)" % (faults, (faults/total)*100))
print("Total # crashes: %d (%f%%)" % (crashes, (crashes/total)*100))